In [25]:
import numpy as np
from gbp.gbp import FactorGraph, VariableNode, Factor

# -----------------------
# SLAM-like base graph
# -----------------------
def make_slam_like_graph(N=100, step_size=25, loop_prob=0.2, loop_radius=50, rng=None):
    if rng is None:
        rng = np.random.default_rng()

    nodes, edges = [], []
    positions = []
    x, y = 0.0, 0.0
    positions.append((x, y))

    # 随机游走生成轨迹
    for _ in range(1, int(N)):
        dx, dy = rng.normal(size=2)
        norm = np.sqrt(dx**2 + dy**2) + 1e-6
        dx, dy = dx / norm * float(step_size), dy / norm * float(step_size)
        x, y = x + dx, y + dy
        positions.append((x, y))

    # 节点
    for i, (px, py) in enumerate(positions):
        nodes.append({
            "data": {"id": f"b{i}", "layer": 0, "dim": 2},
            "position": {"x": float(px), "y": float(py)}
        })

    # 顺序边
    for i in range(int(N) - 1):
        edges.append({"data": {"source": f"b{i}", "target": f"b{i+1}"}})

    # 随机回环
    for i in range(int(N)):
        for j in range(i + 5, int(N)):
            if rng.random() < float(loop_prob):
                xi, yi = positions[i]; xj, yj = positions[j]
                if np.hypot(xi-xj, yi-yj) < float(loop_radius):
                    edges.append({"data": {"source": f"b{i}", "target": f"b{j}"}})
    return nodes, edges


# -----------------------
# GBP Graph 构建
# -----------------------
def build_noisy_pose_graph(
    nodes,
    edges,
    prior_sigma: float = 10,
    odom_sigma: float = 10,
    prior_prop: float = 0.0,
    tiny_prior: float = 1e-11,
    rng=None,
):
    """
    构造二维 pose-only 因子图（线性，高斯），并注入噪声。
    参数:
      prior_sigma : 强先验的标准差（小=强）
      odom_sigma  : 里程计测量噪声标准差
      prior_prop  : 0.0=仅 anchor；(0,1)=按比例随机选；>=1.0=全体
      tiny_prior  : 所有节点默认加的极小先验，防止奇异
    """
    if rng is None:
        rng = np.random.default_rng()

    fg = FactorGraph(nonlinear_factors=False, eta_damping=0)

    # ---- 变量节点 + 先验 ----
    var_nodes = []
    I2 = np.eye(2, dtype=float)
    N = len(nodes)



    # ---- 预生成噪声 ----
    prior_noises = {}
    odom_noises = {}

    # 确定强先验的节点集合
    if prior_prop <= 0.0:
        strong_ids = {0}
    elif prior_prop >= 1.0:
        strong_ids = set(range(N))
    else:
        k = max(1, int(np.floor(prior_prop * N)))
        strong_ids = set(rng.choice(N, size=k, replace=False).tolist())

    # 为 strong prior 节点生成噪声
    for i in strong_ids:
        prior_noises[i] = rng.normal(0.0, prior_sigma, size=2)

    # 为所有边生成噪声
    for e in edges:
        src = e["data"]["source"]; dst = e["data"]["target"]
        if src.startswith("b") and dst.startswith("b"):
            odom_noises[(int(src[1:]), int(dst[1:]))] = rng.normal(0.0, odom_sigma, size=2)
            #


    for i, n in enumerate(nodes):
        v = VariableNode(i, dofs=2)
        # 保存 GT（只用于生成测量 & 初始线性化点）
        v.GT = np.array([n["position"]["x"], n["position"]["y"]], dtype=float)

        # 极小先验（所有节点都有，避免奇异）
        v.prior.lam = tiny_prior * I2
        v.prior.eta = np.zeros(2, dtype=float)
        
        # 强先验（根据 prior_prop 选择）
        if i in strong_ids:
            lam_strong = I2 / (prior_sigma ** 2)
            eta_strong = lam_strong @ (v.GT + prior_noises[i])
            v.prior.lam = v.prior.lam + lam_strong
            v.prior.eta = v.prior.eta + eta_strong

        var_nodes.append(v)

    fg.var_nodes = var_nodes
    fg.n_var_nodes = len(var_nodes)

    # ---- 测量模型（线性的）----
    def meas_fn(xy, *args):
        # measurement = p_j - p_i
        xy = np.asarray(xy, dtype=float)
        return xy[2:] - xy[:2]

    def jac_fn(xy, *args):
        # d(pj - pi)/d[pi,pj] = [-I, I]
        return np.array([[-1, 0, 1, 0],
                         [ 0,-1, 0, 1]], dtype=float)

    # ---- 里程计/回环 因子 ----
    factors = []
    fid = 0
    for e in edges:
        src = e["data"]["source"]; dst = e["data"]["target"]
        # 只连 base 层的节点
        if not (src.startswith("b") and dst.startswith("b")):
            continue
        i = int(src[1:]); j = int(dst[1:])
        vi, vj = var_nodes[i], var_nodes[j]

        # 测量 = GT 差值 + 高斯噪声
        meas = (vj.GT - vi.GT) +odom_noises[(int(src[1:]), int(dst[1:]))]

        f = Factor(fid, [vi, vj], meas, odom_sigma, meas_fn, jac_fn)
        f.type = "base"

        # 用 GT 作为初始线性化点
        linpoint = np.r_[vi.GT, vj.GT]
        f.compute_factor(linpoint=linpoint, update_self=True)

        factors.append(f)
        vi.adj_factors.append(f)
        vj.adj_factors.append(f)
        fid += 1

    fg.factors = factors
    fg.n_factor_nodes = len(factors)
    return fg


# -----------------------
# 主程序：跑 100 次迭代
# -----------------------
if __name__ == "__main__":
    rng = np.random.default_rng(42)  # 固定随机种子

    # 1. 造一个图
    nodes, edges = make_slam_like_graph(N=50, step_size=25, loop_prob=0.1, loop_radius=50, rng=rng)

    # 2. 构建 GBP 图
    fg = build_noisy_pose_graph(nodes, edges, prior_sigma=10.0, odom_sigma=10.0, prior_prop=0.1, rng=rng)

    # 3. 跑迭代
    for it in range(1000):
        fg.synchronous_iteration()
        energy = fg.energy_map(include_priors=True, include_factors=True)
        print(f"Iter {it+1:03d} | Energy = {energy:.6f}")


Iter 001 | Energy = 736.609310
Iter 002 | Energy = 875.120584
Iter 003 | Energy = 1010.759051
Iter 004 | Energy = 613.048495
Iter 005 | Energy = 198.667673
Iter 006 | Energy = 48.870658
Iter 007 | Energy = 34.180278
Iter 008 | Energy = 26.933782
Iter 009 | Energy = 24.075345
Iter 010 | Energy = 23.071541
Iter 011 | Energy = 22.035127
Iter 012 | Energy = 21.511774
Iter 013 | Energy = 21.224143
Iter 014 | Energy = 21.028775
Iter 015 | Energy = 20.874572
Iter 016 | Energy = 20.773651
Iter 017 | Energy = 20.700668
Iter 018 | Energy = 20.652131
Iter 019 | Energy = 20.611422
Iter 020 | Energy = 20.579365
Iter 021 | Energy = 20.555213
Iter 022 | Energy = 20.535346
Iter 023 | Energy = 20.518701
Iter 024 | Energy = 20.504841
Iter 025 | Energy = 20.493142
Iter 026 | Energy = 20.483298
Iter 027 | Energy = 20.474870
Iter 028 | Energy = 20.467617
Iter 029 | Energy = 20.461404
Iter 030 | Energy = 20.456016
Iter 031 | Energy = 20.451352
Iter 032 | Energy = 20.447325
Iter 033 | Energy = 20.443835
Iter

In [24]:
import numpy as np
from gbp.gbp import FactorGraph, VariableNode, Factor

# -----------------------
# SLAM-like base graph
# -----------------------
def make_slam_like_graph(N=100, step_size=25, loop_prob=0.2, loop_radius=50, rng=None):
    if rng is None:
        rng = np.random.default_rng()

    nodes, edges = [], []
    positions = []
    x, y = 0.0, 0.0
    positions.append((x, y))

    # 随机游走生成轨迹
    for _ in range(1, int(N)):
        dx, dy = rng.normal(size=2)
        norm = np.sqrt(dx**2 + dy**2) + 1e-6
        dx, dy = dx / norm * float(step_size), dy / norm * float(step_size)
        x, y = x + dx, y + dy
        positions.append((x, y))

    # 节点
    for i, (px, py) in enumerate(positions):
        nodes.append({
            "data": {"id": f"b{i}", "layer": 0, "dim": 2},
            "position": {"x": float(px), "y": float(py)}
        })

    # 顺序边
    for i in range(int(N) - 1):
        edges.append({"data": {"source": f"b{i}", "target": f"b{i+1}"}})

    # 随机回环
    for i in range(int(N)):
        for j in range(i + 5, int(N)):
            if rng.random() < float(loop_prob):
                xi, yi = positions[i]; xj, yj = positions[j]
                if np.hypot(xi-xj, yi-yj) < float(loop_radius):
                    edges.append({"data": {"source": f"b{i}", "target": f"b{j}"}})
    return nodes, edges




def build_noisy_pose_graph(
    nodes,
    edges,
    prior_sigma: float = 10,
    odom_sigma: float = 10,
    prior_prop: float = 0.0,
    tiny_prior: float = 1e-10,
    rng=None,
):
    if rng is None:
        rng = np.random.default_rng()

    fg = FactorGraph(nonlinear_factors=False, eta_damping=0)

    var_nodes = []
    I2 = np.eye(2, dtype=float)
    N = len(nodes)

    # ---- 预生成噪声 ----
    prior_noises = {}
    odom_noises = {}

    # 确定强先验的节点集合
    if prior_prop <= 0.0:
        strong_ids = {0}
    elif prior_prop >= 1.0:
        strong_ids = set(range(N))
    else:
        k = max(1, int(np.floor(prior_prop * N)))
        strong_ids = set(rng.choice(N, size=k, replace=False).tolist())

    # 为 strong prior 节点生成噪声
    for i in strong_ids:
        prior_noises[i] = rng.normal(0.0, prior_sigma, size=2)

    # 为所有边生成噪声
    for e in edges:
        src = e["data"]["source"]; dst = e["data"]["target"]
        if src.startswith("b") and dst.startswith("b"):
            odom_noises[(int(src[1:]), int(dst[1:]))] = rng.normal(0.0, odom_sigma, size=2)

    # ---- 变量节点 ----
    for i, n in enumerate(nodes):
        v = VariableNode(i, dofs=2)
        v.GT = np.array([n["position"]["x"], n["position"]["y"]], dtype=float)

        # 极小先验
        v.prior.lam = tiny_prior * I2
        v.prior.eta = np.zeros(2, dtype=float)

        var_nodes.append(v)

    fg.var_nodes = var_nodes
    fg.n_var_nodes = len(var_nodes)

    # ---- prior factors ----
    factors = []
    fid = 0
    for i in strong_ids:
        vi = var_nodes[i]
        lam_strong = I2 / (prior_sigma ** 2)
        z = vi.GT + prior_noises[i]

        def meas_fn_unary(x, *args):
            return x
        def jac_fn_unary(x, *args):
            return np.eye(2)

        f = Factor(fid, [vi], z, prior_sigma, meas_fn_unary, jac_fn_unary)
        f.type = "prior"
        f.compute_factor(linpoint=z, update_self=True)

        factors.append(f)
        vi.adj_factors.append(f)
        fid += 1

    # ---- odometry factors ----
    def meas_fn(xy, *args):
        return xy[2:] - xy[:2]
    def jac_fn(xy, *args):
        return np.array([[-1, 0, 1, 0],
                         [ 0,-1, 0, 1]], dtype=float)

    for e in edges:
        src = e["data"]["source"]; dst = e["data"]["target"]
        if not (src.startswith("b") and dst.startswith("b")):
            continue
        i, j = int(src[1:]), int(dst[1:])
        vi, vj = var_nodes[i], var_nodes[j]

        meas = (vj.GT - vi.GT) + odom_noises[(i, j)]

        f = Factor(fid, [vi, vj], meas, odom_sigma, meas_fn, jac_fn)
        f.type = "base"
        linpoint = np.r_[vi.GT, vj.GT]
        f.compute_factor(linpoint=linpoint, update_self=True)

        factors.append(f)
        vi.adj_factors.append(f)
        vj.adj_factors.append(f)
        fid += 1

    fg.factors = factors
    fg.n_factor_nodes = len(factors)
    return fg



# -----------------------
# 主程序：跑 100 次迭代
# -----------------------
if __name__ == "__main__":
    rng = np.random.default_rng(42)  # 固定随机种子

    # 1. 造一个图
    nodes, edges = make_slam_like_graph(N=50, step_size=25, loop_prob=0.1, loop_radius=50, rng=rng)

    # 2. 构建 GBP 图
    fg = build_noisy_pose_graph(nodes, edges, prior_sigma=10.0, odom_sigma=10.0, prior_prop=0.1, rng=rng)

    # 3. 跑迭代
    for it in range(1000):
        fg.synchronous_iteration()
        energy = fg.energy_map(include_priors=True, include_factors=True)
        print(f"Iter {it+1:03d} | Energy = {energy:.6f}")


Iter 001 | Energy = 736.609303
Iter 002 | Energy = 875.120492
Iter 003 | Energy = 1010.758653
Iter 004 | Energy = 613.047975
Iter 005 | Energy = 198.667459
Iter 006 | Energy = 48.870660
Iter 007 | Energy = 34.180285
Iter 008 | Energy = 26.933797
Iter 009 | Energy = 24.075363
Iter 010 | Energy = 23.071562
Iter 011 | Energy = 22.035149
Iter 012 | Energy = 21.511796
Iter 013 | Energy = 21.224165
Iter 014 | Energy = 21.028798
Iter 015 | Energy = 20.874596
Iter 016 | Energy = 20.773675
Iter 017 | Energy = 20.700691
Iter 018 | Energy = 20.652154
Iter 019 | Energy = 20.611446
Iter 020 | Energy = 20.579389
Iter 021 | Energy = 20.555237
Iter 022 | Energy = 20.535371
Iter 023 | Energy = 20.518726
Iter 024 | Energy = 20.504865
Iter 025 | Energy = 20.493166
Iter 026 | Energy = 20.483322
Iter 027 | Energy = 20.474894
Iter 028 | Energy = 20.467642
Iter 029 | Energy = 20.461428
Iter 030 | Energy = 20.456041
Iter 031 | Energy = 20.451377
Iter 032 | Energy = 20.447349
Iter 033 | Energy = 20.443859
Iter